<a href="https://colab.research.google.com/github/BillyTevin11/Coding-Projects/blob/main/Estimation_of_Chlorophyll_a_Concentration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization of Earth Engine

In [1]:
import ee
import geemap
import geemap.colormaps as cm

ee.Authenticate()

ee.Initialize(project="chlaest")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


# Background on Chlorophyll-a

Chlorophyll-a is the primary photosynthetic pigment in most phytoplankton base of many aquatic food webs. The concentration of Chl-a is widely used as a proxy for phytoplankton biomass in water bodies. This is because nearly all phytoplankton contain chl-a, and its concentration correlates well with the total amount of phytoplankton present.

While all algae contain Chl-a, different groups may have varying ratios of Chl-a to other pigments. This can help in identifying and distinguishing between algal groups. Chl-a concentrations in water bodies often show seasonal patterns, reflecting changes in phytoplankton growth due to factors like temperature, light availability, and nutrient levels.

Chl-a's unique spectral signature allows it to be detected by satellite sensors, enabling large-scale monitoring of phytoplankton distributions in oceans and large lakes.

# Study Area

Lake Victoria, Africa's largest lake and the world's second-largest freshwater lake by surface area, spanning approximately 68,800 square kilometers, is situated in East Africa and shared by three countries: Uganda (45%), Tanzania (49%), and Kenya (6%).

Its geographical coordinates range from 0°30'N to 3°12'S and 31°37'E to 34°53'E, encompassing a diverse array of ecological zones and environmental conditions.With a maximum depth of 84 meters and a catchment area of 194,000 km², it supports over 40 million people and hosts over 500 fish species.

The lake faces significant environmental challenges, including eutrophication, invasive species, and climate change impacts. These factors make Lake Victoria an ideal site for chlorophyll-a concentration studies. Such research would aid in monitoring eutrophication, assessing algal blooms, informing fisheries management, and evaluating climate change effects.

The lake's size suits satellite-based estimation techniques, while its transboundary nature necessitates coordinated management. A chlorophyll-a study in Lake Victoria would provide crucial insights for sustainable management of this vital ecosystem, balancing ecological preservation with the needs of the dependent population.

## OSM Basemap Image covering Lake Victoria

In [2]:
Map = geemap.Map(height=675)
Map.setCenter(33.0, -1.35, 8)

# Landsat 8 OLI + TIRS

## Applicability of Landsat 8 in Chl-a Estimation


Previous studies have utilized spectral indices such as the Normalized Difference Chlorophyll Index, NDCI as a proxy to estimate the concentration of Chl-a in inland water bodies. However, this at at times been affected by the presence of numerous vegetation in the water bodies under study and hence difficult to distingush individual vegetation species present in water leading to a subjective Chl-a estimate rather than an objective Chl-a estimate.

The multispectrality of Landsat 8 bands provides several bands which when combined either as indices or even mathematically formulated algorithms yield methods to better map Chl-a concentrations in water bodies.

Landsat 8 is highly applicable for chlorophyll-a estimation studies in Lake Victoria due to its combination of spatial and spectral resolution, regular coverage, and free data availability. Its 30-meter resolution is particularly suited for a large water body like Lake Victoria, allowing researchers to capture both broad patterns and localized phenomena.

The satellite's spectral bands, especially the blue, green, and red bands, align well with the optical properties of chlorophyll-a, enabling the use of various estimation algorithms. The 16-day revisit time provides a good balance between temporal resolution and data volume, suitable for monitoring seasonal changes and long-term trends in chlorophyll-a concentrations.

Landsat 8's global coverage ensures that the entire lake and its basin are captured, facilitating comprehensive studies. The free and open nature of the data makes it cost-effective for long-term monitoring programs, which is crucial for understanding the lake's changing dynamics over time.

## Extracting Lake Victoria's Water Surface

To start, we need to extract the water from Lake Victoria since it's the part of Lake Victoria that concerns us. The shores or surrounding land around Lake Victoria in the estimation and mapping of Chlorophyll-a is insignificant in the study.

To achieve this, we shall use the Modified Normalized Difference Water Index, MNDWI. The MNDWI is a an index in remote sensing used to extract water features especially in built up areas or environments.

To compute MNDWI using Landsat 8, the Green - Band 3 and Short Wave InfraRed - Band 6 bands are utilized. Both have a spatial resolution of 30m. The formula used is as follows:

**MNDWI = (GREEN - SWIR 1) / (GREEN + SWIR 1)**

### Drawing a Geometry Covering Lake Victoria for Subsetting

In [3]:
# Create a geometry object representing the Lake Victoria region.
roi = ee.Geometry.Polygon(
    [[[31.5, 0.5], [35.0, 0.5], [35.0, -3.0], [31.5, -3.0], [31.5, -0.1]]])
Map.centerObject(roi, 8)
Map.addLayer(roi,{},"ROI")
Map

Map(center=[-1.250193135246947, 33.249999999999936], controls=(WidgetControl(options=['position', 'transparent…

### Processing Landsat 8 Data

In [4]:
# Import the Landsat 8 collection.
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-01-01', '2023-12-31') \
    .filterBounds(roi)

In [5]:
# Function to mask clouds and shadows.
def maskL8sr(image):
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

# Apply cloud and shadow masking to the Landsat 8 collection.
l8_masked = l8.map(maskL8sr)

In [6]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [7]:
# Applying the scale factor to the Masked Landsat 8 Images
l8_masked = l8_masked.map(apply_scale_factors)

In [8]:
# Create a median composite of the masked Landsat 8 collection.
l8_median = l8_masked.median().clip(roi)

In [9]:
# Defining Visualization Parameters
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

In [10]:
# Mapping the True Color Imagery of the Defined Geometry, roi Covering Lake Victoria
Map = geemap.Map(height=675)
Map.setCenter(33.0, -1.35, 8)
Map.addLayer(l8_median, visualization, 'Landsat 8 True Color Image - 2023',False)

### Computing MNDWI

In [18]:
# Compute MNDWI.
mndwi = l8_median.normalizedDifference(['SR_B3', 'SR_B6'])

palette = cm.palettes.ndwi

# Define visualization parameters for MNDWI.
vis_params = {
    'min': -1,
    'max': 1,
    'palette': palette
}

# Add the MNDWI layer to the map.
Map.addLayer(mndwi, vis_params, 'MNDWI',False)
Map.add_colorbar(vis_params, label="MNDWI", layer_name="MNDWI",position='bottomleft')

In [12]:
# Threshold the MNDWI image to identify water pixels. Adjust threshold as needed
water_mask = mndwi.gt(-0.3)

# Create a binary image where water is 1 and non-water is 0.
binary_water_mask = water_mask.where(water_mask, 1).where(water_mask.Not(), 0)

# Define a palette where water is blue
palette = ['blue']

# Extract water pixels where the bit value is 1.
water_pixels = l8_median.updateMask(binary_water_mask.eq(1))

# Select the first band for visualization
water_pixels_vis = water_pixels.select(0) # Selecting the first band (index 0)

# Add the water pixels layer to the map.
Map.addLayer(water_pixels_vis, {'palette': palette}, 'Lake Victoria',False)

### Estimating Chlorophyll-a using NDCI as a Proxy

The Normalized Difference Chlorophyll Index, NDCI is used frequently in remote sensing as a proxy for chlorophyll-a in water bodies.

In [17]:
# Compute the NDCI solely covering Lake Victoria

# Compute NDCI using the water pixels image
ndci = water_pixels.normalizedDifference(['SR_B5', 'SR_B4'])

palette = cm.palettes.brg
# Define visualization parameters for NDCI.
ndci_vis = {
    'min': -1,
    'max': 1,
    'palette': palette
}

# Add the NDCI layer to the map.
Map.addLayer(ndci, ndci_vis, 'NDCI',False)
Map.add_colorbar(ndci_vis, label="NDCI", layer_name="NDCI",position='bottomleft')

### Estimation of Chl-a


The Formula for Estimating Chlorophyll-a was derived from [Nguyen Trinh Duc Hieu et al 2023 ](https://iopscience.iop.org/article/10.1088/1755-1315/1226/1/012010)

                        
                        Chl-a = 0.04 e^(3.15 RrsB3/RrsB2 )

In [14]:
# Define a function to calculate Chl-a using the formula
def calculate_chla(image):
  # Select the bands needed for the formula
  band_b2 = image.select('SR_B2')
  band_b3 = image.select('SR_B3')

  # Calculate RrsB3/RrsB2
  ratio = band_b3.divide(band_b2)

  # Calculate Chl-a using image.expression() to apply exp() pixel-wise
  chla_image = image.expression(
      '0.04 * exp(ratio * 3.15)',
      {'ratio': ratio}
  ).rename('Chl-a')

  # Return the image with the Chl-a band added.
  return image.addBands(chla_image) # Add the chla_image instead of the chla Number

In [15]:
# Apply the function to the water pixels image
chla_image = calculate_chla(water_pixels)

In [19]:
palette = cm.palettes.brg

# Define visualization parameters for Chl-a
chla_vis = {
    'min': 0,
    'max': 6.14e+306,
    'palette': palette
}

# Add Chl-a layer to the map
Map.addLayer(chla_image.select('Chl-a'), chla_vis, 'Chlorophyll-a')
Map.add_colorbar(chla_vis, label="Chl-a (µg/L)", layer_name="Chlorophyll-a",position='bottomleft')
Map

Map(bottom=33351.0, center=[-1.35, 33.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…